dataset link: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=car+details+v4.csv

In [1]:
pip install "git+https://github.com/emanuelegaliano/TRACCIA.git" # type: ignore

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emanuelegaliano/TRACCIA.git to /tmp/pip-req-build-mc80nqda
  Running command git clone --filter=blob:none --quiet https://github.com/emanuelegaliano/TRACCIA.git /tmp/pip-req-build-mc80nqda
  Resolved https://github.com/emanuelegaliano/TRACCIA.git to commit c95f551a16b27ed738e919564f3b7466b1d70695
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_csv("dataset/car_details_v4.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Make                2059 non-null   object 
 1   Model               2059 non-null   object 
 2   Price               2059 non-null   int64  
 3   Year                2059 non-null   int64  
 4   Kilometer           2059 non-null   int64  
 5   Fuel Type           2059 non-null   object 
 6   Transmission        2059 non-null   object 
 7   Location            2059 non-null   object 
 8   Color               2059 non-null   object 
 9   Owner               2059 non-null   object 
 10  Seller Type         2059 non-null   object 
 11  Engine              1979 non-null   object 
 12  Max Power           1979 non-null   object 
 13  Max Torque          1979 non-null   object 
 14  Drivetrain          1923 non-null   object 
 15  Length              1995 non-null   float64
 16  Width 

In [3]:
from traccia.routes.data_miner import (
    DataMiningFootprint,
    VariableCleaningFootstep,
    MissingValuesFootstep,
    PreprocessingPath,
)

print("Original columns:")
print(list(df.columns))

# 2. Crea il footprint
footprint = DataMiningFootprint(raw_df=df)

# 3. Configura il VariableCleaningFootstep
rename_map = {
    "Make": "make",
    "Model": "model",
    "Price": "price",
    "Year": "year",
    "Kilometer": "kilometers",
    "Fuel Type": "fuel_type",
    "Transmission": "transmission",
    "Location": "location",
    "Color": "color",
    "Owner": "owner",
    "Seller Type": "seller_type",
}

expected_dtypes = {
    "price": "float64",       # comodo averlo come float
    "year": "int64",
    "kilometers": "int64",
    "fuel_type": "string",
    "transmission": "string",
    "location": "string",
    "color": "string",
    "owner": "string",
    "seller_type": "string",
}

cleaning_step = VariableCleaningFootstep(
    rename_map=rename_map,
    expected_dtypes=expected_dtypes,
    strict=False,  # per ora log, non crash
)

# 4. Configura il MissingValuesFootstep
missing_step = MissingValuesFootstep(
    column_strategies={
        "price": "median",       # numerica → riempi con mediana
        "kilometers": "mean",    # numerica → riempi con media
        "fuel_type": "mode",     # categorica → moda
        "owner": "constant",     # categorica → costante
    },
    constant_values={
        "owner": "Unknown",
    },
    strict=False,  # logga problemi, non crasha
)

# 5. Crea il Path di preprocessing
preprocess_path = PreprocessingPath(
    cleaning_step=cleaning_step,
    missing_step=missing_step,
    run_id="cars_preprocessing_v1",
)

# 6. Esegui il Path (chiama solo il primo footstep, la chain fa il resto)
footprint = preprocess_path.run(footprint)

# 7. Ispeziona il risultato
print("\n=== dtypes after preprocessing ===")
print(footprint.cleaned_df.dtypes)

print("\n=== head of cleaned_df ===")
print(footprint.cleaned_df.head())

print("\n=== preprocessing log ===")
for line in footprint.preprocessing_log:
    print(" -", line)

print("\n=== metadata ===")
print(footprint.get_metadata().to_dict())

ImportError: cannot import name 'MissingValuesFootstep' from 'traccia.routes.data_miner' (/home/manu/.local/lib/python3.14/site-packages/traccia/routes/data_miner/__init__.py)